### קוד מבוא

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [4]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [5]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [6]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [7]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [8]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [9]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


### העלת טבלת המרה בין למס מרכזי למשני

In [95]:
stat_join_from_main_to_secondary=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output','stat_join_from_main_to_secondary')

### העלת טבלת יח_ד למ_ס

In [96]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\מרשם דירות\2022'
file_name='מספר דירות לפי אזורים סטטיסטיים'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')


stat_aprt=df

In [97]:
col=['STAT', 'aprt_20']

In [98]:
stat_aprt=stat_aprt[col]

### שכבת אזורים סטטיסטים

In [99]:
path=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\אזורים סטטיסטים\statisticalareas_2020_demography.gdb'
layer_list=fiona.listlayers(path)

layer_name='statisticalareas_2020_demography_jtmt_area'

gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name)).fillna(0)

stat=gpd_layer.rename(columns={'YISHUV_STAT11':'STAT'})

In [100]:
col_name=['STAT','geometry']
stat=stat[col_name]

In [101]:
stat=pd.merge(stat,stat_aprt,on='STAT',how='left').merge(stat_join_from_main_to_secondary,left_on='STAT',right_on='secondary_stat',how='left')

In [102]:
stat.loc[stat['precent_of_stat_data'].isna(),'main_stat']=stat['STAT']

In [103]:
stat.loc[stat['precent_of_stat_data'].isna(),'secondary_stat']=stat['STAT']

In [104]:
stat.loc[stat['precent_of_stat_data'].isna(),'precent_of_stat_data']=1

###  מידע ברמת רשות

In [105]:
stat_point=make_point(stat[['STAT', 'geometry']])

In [106]:
muni_under_JTMT_ITM=up_load_gdb(r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\משרד הפנים\MUNI_border.gdb','muni_under_JTMT_ITM')

In [107]:
muni_under_JTMT_ITM=muni_under_JTMT_ITM[['CR_PNIM','geometry']]

In [108]:
stat=stat.set_index('STAT')

In [109]:
stat['CR_PNIM']=stat_point.sjoin(muni_under_JTMT_ITM)[['STAT','CR_PNIM']].set_index('STAT')

In [110]:
stat=stat.reset_index()

###  העלת מידע אוכלוסייה

In [111]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם'
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='classification')
df=df.dropna(how='all')


classification=df

In [112]:
col_name=['main_stat','classification_name']
classification=classification[col_name]

In [113]:
classification=classification.drop_duplicates(subset='main_stat',keep='first')

In [114]:
stat=pd.merge(stat,classification,on='main_stat',how='left')

In [115]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם'
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='hh_size')
df=df.dropna(how='all')


hh_size=df

In [116]:
stat=pd.merge(stat,hh_size,on='classification_name',how='left')

In [10]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם'
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='types')
df=df.dropna(how='all')


age_des_types=df

age_des_types.pivot_table(columns='age_name',index='classification_name',aggfunc=sum,values='perc').to_excel(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם\התפלגות_גילים_קטלוג\age_des_types_230719.xlsx')

In [118]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם'
file_name='cbs_2020_with_age_distribution_type'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df,sheet_name='absolute_numbers')
df=df.dropna(how='all').fillna(0)


cbs_pop=df

In [119]:
cbs_pop['pop']=cbs_pop.iloc[:,3:].sum(axis=1)

In [120]:
age=['0','5','10','15','20','25','30','35','40','45','50','55','60','65','70','75','80','85']

In [121]:
for x in age:
    cbs_pop['pop_{}'.format(x)]= cbs_pop['female_{}'.format(x)]+cbs_pop['male_{}'.format(x)]

In [122]:
cbs_pop['pop_75up']= cbs_pop[['pop_75', 'pop_80', 'pop_85']].sum(axis=1)

In [123]:
col=[ 'main_stat', 'pop', 'pop_0', 'pop_5', 'pop_10', 'pop_15', 'pop_20', 'pop_25', 'pop_30', 'pop_35', 'pop_40', 'pop_45', 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [124]:
cbs_pop=cbs_pop[col]

יצירת טבלת התפלגות גילים באחוזים לטובת המשך הקוד כאשר יש שינויים בסך הכל אוכלוסיה אבל מעוניינים עדיין בהתפלגות גילים המקורית

In [125]:
cbs_pop_pre=cbs_pop.copy()

In [126]:
col=list(cbs_pop_pre.iloc[:,1:])

In [127]:
cbs_pop_pre.loc[:, col] = cbs_pop_pre.loc[:, col].div(cbs_pop_pre['pop'], axis=0)

In [128]:
cbs_pop_pre=cbs_pop_pre.fillna(0)

In [129]:
cbs_pop_pre=cbs_pop_pre.rename(columns={'pop':'pop_pre'})

In [130]:
cbs_pop_pre=cbs_pop_pre.merge(cbs_pop[['main_stat', 'pop']],on='main_stat',how='left')

In [131]:
stat=pd.merge(stat,cbs_pop_pre,on='main_stat',how='left')

In [132]:
stat['pop']=stat['pop']*stat['precent_of_stat_data']

###  אחוז חרדים מכלל א"ס

In [133]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\חרדים\2020'
file_name='אוכלוסייה חרדית לפי מחוז, יישוב, ואזור סטטיסטי - 2020'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')


stat_hardi=df

In [134]:
stat_hardi=stat_hardi[['pre_hardi','main_stat']].set_index('main_stat')

In [135]:
stat=stat.set_index('main_stat')

In [136]:
stat['pre_hardi']=stat_hardi['pre_hardi']

In [137]:
stat['pre_hardi']=stat['pre_hardi'].fillna(0)

###  תיקון שלב א ערבי ירושלים

In [138]:
path=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\אזורים סטטיסטים\statisticalareas_2020_demography.gdb'
layer_name='statisticalareas_2020_demography_arab_jtmt'
stat_arab=up_load_gdb(path,layer_name)

In [139]:
col=['STAT', 'group_name']

In [140]:
stat_arab=stat_arab[col]

In [141]:
stat_arab=stat_arab.merge(cbs_pop_pre,left_on='STAT',right_on='main_stat',how='left')

In [142]:
stat_arab=stat_arab.set_index('group_name')

In [143]:
group_pop=stat_arab.groupby(by='group_name').sum()[['pop']]

In [144]:
stat_arab['group_pop']=group_pop['pop']

In [145]:
stat_arab['pre_from_group_pop']=stat_arab['pop']/stat_arab['group_pop']

In [146]:
data = [['akev', 55000], ['Shuafat', 75000],['east_jeru_left',0]]

In [147]:
group_pop_jtmt = pd.DataFrame(data, columns=['group_name', 'pop']).set_index('group_name')

In [148]:
group_pop_delta=group_pop-group_pop_jtmt

In [149]:
group_pop_delta=group_pop_delta.loc['east_jeru_left']+(group_pop_delta.loc['Shuafat']+group_pop_delta.loc['akev'])*0.8#בגלל שהדלתא מוסבר עי הגירה מיוש ולא רק מהעיר ירושלים 

In [150]:
group_pop_jtmt.loc['east_jeru_left','pop']=group_pop_delta.item()

In [151]:
stat_arab['group_pop_jtmt']=group_pop_jtmt['pop']

In [152]:
stat_arab['pop']=stat_arab['pre_from_group_pop']*stat_arab['group_pop_jtmt']

###  הטמעת תיקון שלב א

In [153]:
stat_arab=stat_arab.set_index('main_stat')

In [154]:
stat['pop_cbs']=stat['pop']

In [155]:
stat.loc[list(stat_arab.index),'pop']=stat_arab['pop']

In [156]:
stat['change_from_cbs']=''

In [157]:
stat.loc[list(stat_arab.index),'change_from_cbs']='| general_arab_change |'

###  העלת א"ס בעייתי לתיקון שלב ב

In [159]:
path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם\jtmt_fix_for_cbs_data_2020.shp'

jtmt_fix_stat=up_load_shp(path)

In [160]:
col=['STAT',
 'fix_pop',
 'fix_aprt',
 'fix_class']

In [161]:
jtmt_fix_stat=jtmt_fix_stat[col].set_index('STAT')

In [162]:
stat=stat.set_index('STAT')

In [163]:
stat=stat.join(jtmt_fix_stat, how='left')

###  יצירת גמ לפי קטלוג

In [164]:
cbs_not_lie_lst=list(stat.query('fix_pop.isna() & fix_aprt.isna() & fix_class.isna() ').index)

In [165]:
stat['count']=1

In [166]:
stat=stat.reset_index()

In [167]:
stat.loc[stat['STAT'].isin(cbs_not_lie_lst),['fix_pop','fix_aprt','fix_class']]=1

In [168]:
hh_size_by_classification=stat.loc[(stat['aprt_20']>0)&(stat['STAT'].isin(cbs_not_lie_lst))].pivot_table(index='classification_name',aggfunc=sum)[['aprt_20','pop','count']]

In [169]:
hh_size_by_classification['hh_size']=hh_size_by_classification['pop']/hh_size_by_classification['aprt_20']

In [170]:
hh_size_by_classification=hh_size_by_classification[['hh_size','count']]

In [171]:
hh_size=hh_size.merge(hh_size_by_classification.reset_index(),how='left',on='classification_name',suffixes=('','_cbs'))

###  תיקון שלב ב

In [172]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==1)&(stat['fix_class']==1),'pop']=stat['aprt_20']*stat['hh_size']

In [173]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==1)&(stat['fix_class']==1),'change_from_cbs']=stat['change_from_cbs']+'| pop_created_from_cbs_aprt |'

In [174]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'pop']=0

In [175]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'change_from_cbs']=stat['change_from_cbs']+'| cbs_pop_deleted |'

In [176]:
stat['aprt_20_cbs']=stat['aprt_20']

In [177]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'aprt_20']=0

In [178]:
stat.loc[(stat['fix_pop']==0)&(stat['fix_aprt']==0)&(stat['fix_class']==0),'change_from_cbs']=stat['change_from_cbs']+'| cbs_aprt_deleted |'

In [179]:
stat.loc[(stat['fix_pop']==1)&(stat['fix_aprt']==0)&(stat['fix_class']==1),'aprt_20']=stat['pop']/stat['hh_size']

In [180]:
stat.loc[(stat['fix_pop']==1)&(stat['fix_aprt']==0)&(stat['fix_class']==1),'change_from_cbs']=stat['change_from_cbs']+'| aprt_created_from_pop |'

כאן מתקנים מספר דירות לפי גודל משק בית באזורים ערביים שהגודל משק בית יותר או פחות 20% ממה שאנחנו חושבים שצריך להיות שם

In [181]:
stat.loc[(stat['classification_name'].str.contains("ערבי")==True)&(((stat['pop']/stat['aprt_20'])/stat['hh_size']>1.2)|((stat['pop']/stat['aprt_20'])/stat['hh_size']<0.8)),'aprt_20']=stat['pop']/stat['hh_size']

In [182]:
stat.loc[(stat['classification_name'].str.contains("ערבי")==True)&(((stat['pop']/stat['aprt_20'])/stat['hh_size']>1.2)|((stat['pop']/stat['aprt_20'])/stat['hh_size']<0.8)),'change_from_cbs']=stat['change_from_cbs']+'| aprt_created_from_pop_because_hh_size_not_in_range |'

###  יצירת דירות לפי גודל משק בית

In [183]:
stat.loc[((stat['aprt_20'].isna())|(stat['aprt_20']==0))&(stat['pop']>0),'change_from_cbs']=stat['change_from_cbs']+'| aprt_created_from_pop_because_no_cbs_data |'

In [184]:
stat.loc[((stat['aprt_20'].isna())|(stat['aprt_20']==0))&(stat['pop']>0),'aprt_20']=round(stat['pop']/stat['hh_size'])

### בקרת מידע למס ברמה של רשות


In [185]:
pop_2020_cbs_muni=up_load_df(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\חומר גלם\pop_cbs_muni_2020','pop_2020_cbs_muni')

In [186]:
stat['CR_PNIM']=stat['CR_PNIM'].fillna(0).astype(int)

In [187]:
stat_by_muni_sum=stat.pivot_table(index='CR_PNIM',aggfunc=sum)[['pop_cbs','pop']]

In [188]:
pop_2020_cbs_muni=pop_2020_cbs_muni.set_index('CR_PNIM')

In [189]:
pop_2020_cbs_muni.join(stat_by_muni_sum,how='inner')

,muni_name,disc,pop_cbs_muni_total,pop_cbs,pop
CR_PNIM,,,,,
72,שומרון,אזור יהודה והשומרון,47244,"50,499.00","50,499.00"
73,מטה בנימין,אזור יהודה והשומרון,71630,"68,526.00","68,526.00"
74,מגילות ים המלח,אזור יהודה והשומרון,1968,"1,844.00","1,844.00"
75,ערבות הירדן,אזור יהודה והשומרון,5767,"5,100.00","5,100.00"
76,גוש עציון,אזור יהודה והשומרון,24936,"24,517.00","24,517.00"
78,הר חברון,אזור יהודה והשומרון,9419,"8,934.00","8,934.00"
472,אבו גוש,ירושלים,7771,"7,772.00","7,772.00"
1015,מבשרת ציון,ירושלים,24247,"24,251.00","24,251.00"
1137,קריית יערים,ירושלים,6238,"6,237.00","6,237.00"


### ייצא מידע ברמת אזור סטט עם השינוים שלנו

In [190]:
stat=stat.fillna(0)

In [191]:
stat['pop_delta']=stat['pop']-stat['pop_cbs']

In [192]:
stat['aprt_20_delta']=stat['aprt_20']-stat['aprt_20_cbs']

In [193]:
col=['STAT','precent_of_stat_data','classification_name','pop','pop_cbs','aprt_20','aprt_20_cbs','pop_delta','aprt_20_delta','change_from_cbs']

In [194]:
stat[col].query('STAT!=5526 & STAT!=9975').to_excel(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית\stat_cbs_jtmt_2020_short.xlsx',index=False)

In [195]:
stat.query('STAT!=5526 & STAT!=9975').to_excel(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2020\מרכיבי_תחזית\stat_cbs_jtmt_2020.xlsx',index=False)

path=r'\\FILE-SRV\Jtmt\projections_team\GIS_backround\INFO\למ_ס\אזורים סטטיסטים\statisticalareas_2020_demography.gdb'
layer_list=fiona.listlayers(path)

layer_name='statisticalareas_2020_demography_jtmt_area'

gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name)).fillna(0)

stat_border=gpd_layer.rename(columns={'YISHUV_STAT11':'STAT'})

stat_border[['geometry','STAT']].merge(stat[col],on='STAT',how='right').query('STAT!=5526 & STAT!=9975').to_file(r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output\stat_cbs_jtmt_2020.shp',encoding='utf-8')